In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

/Users/abbieschindler/opt/anaconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


### Arduino data

#### Get paths for files to process

In [2]:
#create list of dir paths
arduino_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r2.intruders/arduino'
    
arduino_file_names = os.listdir(arduino_path)

arduino_data_dir_paths = []

for file in arduino_file_names:
    int_path = arduino_path + '/' + file
    arduino_data_dir_paths.append(int_path)
    
arduino_data_dir_paths

['/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r2.intruders/arduino/arduino_210616.csv']

#### Clean df and combine into 1

In [3]:
data_arduino = pd.DataFrame()

for path in arduino_data_dir_paths:
    print(path)
    
    if path.split('/')[-1] == '.DS_Store':
        continue
        
    #create data table
    data_arduino_int = pd.read_csv(path)
    data_arduino_int = pd.DataFrame(data = data_arduino_int)
    data_arduino_int = data_arduino_int.drop(['Unnamed: 0'], axis=1)

    print(data_arduino_int.shape)
    
    #clean up and create datetime columns
    data_arduino_int['arduino_datetime'] = pd.to_datetime(data_arduino_int['timestamp'])

    data_arduino_int['arduino_date'] = data_arduino_int['arduino_datetime'].dt.date
    data_arduino_int['arduino_day'] = data_arduino_int['arduino_datetime'].dt.day
    data_arduino_int['arduino_time'] = data_arduino_int['arduino_datetime'].dt.strftime("%H:%M:%S")
    data_arduino_int['arduino_hour'] = data_arduino_int['arduino_datetime'].dt.hour
    data_arduino_int['arduino_min'] = data_arduino_int['arduino_datetime'].dt.minute
    data_arduino_int['arduino_second'] = data_arduino_int['arduino_datetime'].dt.second

    if data_arduino.empty:
        data_arduino = data_arduino_int
    else:
        data_arduino = pd.concat([data_arduino, data_arduino_int], axis=0)
        
    print(data_arduino.shape,'\n')
    
data_arduino.head()

/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r2.intruders/arduino/arduino_210616.csv
(23490, 3)
(23490, 10) 



,timestamp,device,signal,arduino_datetime,arduino_date,arduino_day,arduino_time,arduino_hour,arduino_min,arduino_second
0,2021-06-16 08:14:56,A3,1023.0,2021-06-16 08:14:56,2021-06-16,16,08:14:56,8,14,56
1,2021-06-16 08:14:56,A3,1023.0,2021-06-16 08:14:56,2021-06-16,16,08:14:56,8,14,56
2,2021-06-16 08:14:56,A3,1023.0,2021-06-16 08:14:56,2021-06-16,16,08:14:56,8,14,56
3,2021-06-16 08:14:56,A3,1023.0,2021-06-16 08:14:56,2021-06-16,16,08:14:56,8,14,56
4,2021-06-16 08:14:56,A3,1023.0,2021-06-16 08:14:56,2021-06-16,16,08:14:56,8,14,56


In [4]:
data_arduino = data_arduino.groupby(['timestamp', 'arduino_datetime', 'arduino_date', 'arduino_time', 'device']).max().reset_index()
print(data_arduino.shape)
data_arduino.head()

(1805, 10)


,timestamp,arduino_datetime,arduino_date,arduino_time,device,signal,arduino_day,arduino_hour,arduino_min,arduino_second
0,2021-06-16 08:14:56,2021-06-16 08:14:56,2021-06-16,08:14:56,A3,1023.0,16,8,14,56
1,2021-06-16 08:14:57,2021-06-16 08:14:57,2021-06-16,08:14:57,A3,1023.0,16,8,14,57
2,2021-06-16 08:14:58,2021-06-16 08:14:58,2021-06-16,08:14:58,A3,1023.0,16,8,14,58
3,2021-06-16 08:14:59,2021-06-16 08:14:59,2021-06-16,08:14:59,A3,1023.0,16,8,14,59
4,2021-06-16 08:15:02,2021-06-16 08:15:02,2021-06-16,08:15:02,A3,1023.0,16,8,15,2


In [ ]:
data_arduino[data_arduino['device']=='A1']['timestamp'].value_counts()
#data_arduino[(data_arduino['device']=='A1') & (data_arduino['timestamp']=='2021-06-16 19:17:19')]
arduino_count = data_arduino.groupby(['device', 'timestamp', 'arduino_day', 'arduino_hour', 'arduino_min', 'arduino_second'])['signal'].count().reset_index().sort_values(['timestamp'])
arduino_count.head(100)
arduino_count.groupby(['device']).max().reset_index()

In [ ]:
data_int = arduino_count[arduino_count['arduino_day'] == 22]
data_int = data_int[data_int['arduino_hour'] ==19]
data_int = data_int[data_int['arduino_min'] ==34]
groupby = data_int.groupby(['device', 'arduino_second'])['signal'].sum().reset_index()
#groupby = data_int.groupby(['Injury', 'IdRFID', 'variable'])['value'].sum().reset_index().groupby(['Injury', 'variable'])['value'].mean().reset_index()
    #viz
groupby = groupby.pivot('device', 'arduino_second', "signal")
plt.figure(figsize=(15,5))
ax = sns.heatmap(groupby, cmap="Reds", vmin=0, vmax=120, cbar_kws={'label': 'Total intake (ml)'})
    #plt.xlabel('Bottle')
plt.show()

#### Add day, hour, and min counts

In [5]:
#column of running day count
dat_map = dict(zip(data_arduino['arduino_date'].unique(), np.arange(1,len(data_arduino['arduino_date'].unique())+1)))
data_arduino['arduino_day_count'] = data_arduino['arduino_date'].map(dat_map)

#column of running min and second count within each day
data_arduino['arduino_min_count'] = (data_arduino['arduino_hour']*60) + data_arduino['arduino_min'] 
data_arduino['arduino_sec_count'] = (data_arduino['arduino_min_count']*60) + data_arduino['arduino_second'] 

#column of running hour and minute count across all days
data_arduino['arduino_hour_count_running'] = (data_arduino['arduino_day_count']*24) + data_arduino['arduino_hour'] - 24
data_arduino['arduino_min_count_running'] = (data_arduino['arduino_hour_count_running']*60) + data_arduino['arduino_min'] 
data_arduino['arduino_sec_count_running'] = (data_arduino['arduino_min_count_running']*60) + data_arduino['arduino_second'] - 60

print(data_arduino.shape)
data_arduino.head()

(1805, 16)


,timestamp,arduino_datetime,arduino_date,arduino_time,device,signal,arduino_day,arduino_hour,arduino_min,arduino_second,arduino_day_count,arduino_min_count,arduino_sec_count,arduino_hour_count_running,arduino_min_count_running,arduino_sec_count_running
0,2021-06-16 08:14:56,2021-06-16 08:14:56,2021-06-16,08:14:56,A3,1023.0,16,8,14,56,1,494,29696,8,494,29636
1,2021-06-16 08:14:57,2021-06-16 08:14:57,2021-06-16,08:14:57,A3,1023.0,16,8,14,57,1,494,29697,8,494,29637
2,2021-06-16 08:14:58,2021-06-16 08:14:58,2021-06-16,08:14:58,A3,1023.0,16,8,14,58,1,494,29698,8,494,29638
3,2021-06-16 08:14:59,2021-06-16 08:14:59,2021-06-16,08:14:59,A3,1023.0,16,8,14,59,1,494,29699,8,494,29639
4,2021-06-16 08:15:02,2021-06-16 08:15:02,2021-06-16,08:15:02,A3,1023.0,16,8,15,2,1,495,29702,8,495,29642


#### Add light dark

In [6]:
def assign_ld(time):
    if (time >= 6) & (time < 18):
        return 'light'
    if (time >= 0) & (time < 6):
        return 'dark_am'
    if (time >= 18):
        return 'dark_pm'
    
data_arduino['ld_seg'] = data_arduino['arduino_hour'].apply(assign_ld)

print(data_arduino.shape)
data_arduino.head()


(1805, 17)


,timestamp,arduino_datetime,arduino_date,arduino_time,device,signal,arduino_day,arduino_hour,arduino_min,arduino_second,arduino_day_count,arduino_min_count,arduino_sec_count,arduino_hour_count_running,arduino_min_count_running,arduino_sec_count_running,ld_seg
0,2021-06-16 08:14:56,2021-06-16 08:14:56,2021-06-16,08:14:56,A3,1023.0,16,8,14,56,1,494,29696,8,494,29636,light
1,2021-06-16 08:14:57,2021-06-16 08:14:57,2021-06-16,08:14:57,A3,1023.0,16,8,14,57,1,494,29697,8,494,29637,light
2,2021-06-16 08:14:58,2021-06-16 08:14:58,2021-06-16,08:14:58,A3,1023.0,16,8,14,58,1,494,29698,8,494,29638,light
3,2021-06-16 08:14:59,2021-06-16 08:14:59,2021-06-16,08:14:59,A3,1023.0,16,8,14,59,1,494,29699,8,494,29639,light
4,2021-06-16 08:15:02,2021-06-16 08:15:02,2021-06-16,08:15:02,A3,1023.0,16,8,15,2,1,495,29702,8,495,29642,light


#### Add day/night mapping (lines up dates and ld segments)

In [7]:
ld_mapping_path = '/Users/abbieschindler/Documents/ProgrammingFun/iPythonScripts/ld_mapping.xlsx'

ld_mapping = pd.read_excel(ld_mapping_path)
ld_mapping = pd.DataFrame(data = ld_mapping)
print(ld_mapping.shape)
ld_mapping.head()

(89, 3)


,day,ld_seg,ld
0,2,dark_am,dark_night1
1,3,dark_am,dark_night2
2,4,dark_am,dark_night3
3,5,dark_am,dark_night4
4,6,dark_am,dark_night5


In [8]:
data_arduino = data_arduino.merge(ld_mapping, right_on=['day', 'ld_seg'], left_on=['arduino_day_count', 'ld_seg'])
print(data_arduino.shape)
data_arduino.head()

(1805, 19)


,timestamp,arduino_datetime,arduino_date,arduino_time,device,signal,arduino_day,arduino_hour,arduino_min,arduino_second,arduino_day_count,arduino_min_count,arduino_sec_count,arduino_hour_count_running,arduino_min_count_running,arduino_sec_count_running,ld_seg,day,ld
0,2021-06-16 08:14:56,2021-06-16 08:14:56,2021-06-16,08:14:56,A3,1023.0,16,8,14,56,1,494,29696,8,494,29636,light,1,light_day1
1,2021-06-16 08:14:57,2021-06-16 08:14:57,2021-06-16,08:14:57,A3,1023.0,16,8,14,57,1,494,29697,8,494,29637,light,1,light_day1
2,2021-06-16 08:14:58,2021-06-16 08:14:58,2021-06-16,08:14:58,A3,1023.0,16,8,14,58,1,494,29698,8,494,29638,light,1,light_day1
3,2021-06-16 08:14:59,2021-06-16 08:14:59,2021-06-16,08:14:59,A3,1023.0,16,8,14,59,1,494,29699,8,494,29639,light,1,light_day1
4,2021-06-16 08:15:02,2021-06-16 08:15:02,2021-06-16,08:15:02,A3,1023.0,16,8,15,2,1,495,29702,8,495,29642,light,1,light_day1


#### Map unitLabel and bottle contents

In [9]:
unitLabel_map = {'A0': 'arduino_EtOH10',
                 'A1': 'arduino_Suc2',
                 'A2': 'arduino_EtOH20',
                 'A3': 'arduino_H20'}

data_arduino['unitLabel_drink'] = data_arduino['device'].map(unitLabel_map)
print(data_arduino.shape)
data_arduino.head()

(1805, 20)


,timestamp,arduino_datetime,arduino_date,arduino_time,device,signal,arduino_day,arduino_hour,arduino_min,arduino_second,arduino_day_count,arduino_min_count,arduino_sec_count,arduino_hour_count_running,arduino_min_count_running,arduino_sec_count_running,ld_seg,day,ld,unitLabel_drink
0,2021-06-16 08:14:56,2021-06-16 08:14:56,2021-06-16,08:14:56,A3,1023.0,16,8,14,56,1,494,29696,8,494,29636,light,1,light_day1,arduino_H20
1,2021-06-16 08:14:57,2021-06-16 08:14:57,2021-06-16,08:14:57,A3,1023.0,16,8,14,57,1,494,29697,8,494,29637,light,1,light_day1,arduino_H20
2,2021-06-16 08:14:58,2021-06-16 08:14:58,2021-06-16,08:14:58,A3,1023.0,16,8,14,58,1,494,29698,8,494,29638,light,1,light_day1,arduino_H20
3,2021-06-16 08:14:59,2021-06-16 08:14:59,2021-06-16,08:14:59,A3,1023.0,16,8,14,59,1,494,29699,8,494,29639,light,1,light_day1,arduino_H20
4,2021-06-16 08:15:02,2021-06-16 08:15:02,2021-06-16,08:15:02,A3,1023.0,16,8,15,2,1,495,29702,8,495,29642,light,1,light_day1,arduino_H20


#### Viz

In [ ]:
#data_RFID_viz = data_RFID_final[data_RFID_final['unitLabel_drink'] == 'RFID_H20']
#data_RFID_viz = data_RFID_final[data_RFID_final['RFID_min_count_running'] == 506]

#data_arduino_viz = data_arduino[data_arduino['arduino_min_count_running'] == 506]
#data_VDM_viz = data_VDM_final[data_VDM_final['min_count_running'] == 506]

colorCodes = ['C{}'.format(i) for i in range(4)]

data_1 = data_arduino[data_arduino['device'] == 'A0']['arduino_sec_count_running'].values
data_2 = data_arduino[data_arduino['device'] == 'A1']['arduino_sec_count_running'].values
data_3 = data_arduino[data_arduino['device'] == 'A2']['arduino_sec_count_running'].values
data_4 = data_arduino[data_arduino['device'] == 'A3']['arduino_sec_count_running'].values

data_array = np.array([data_1, data_2, data_3, data_4])

# Draw a spike raster plot
plt.figure(figsize=(50,15))
params = {'axes.labelsize': 75,
          'axes.titlesize': 75}
plt.rc('xtick', labelsize=40) 
plt.rc('ytick', labelsize=40) 
plt.rcParams.update(params)
plt.yticks([0,1,2,3], 
           ['A0', 'A1', 'A2', 'A3'])
plt.eventplot(data_array, color=colorCodes, linelengths = 0.3) 
plt.xlabel('Time (running second count)')
plt.ylabel('Animal and group number')


plt.show()

### RFID data

#### Get paths for files to process

In [11]:
#create list of dir paths
RFID_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/RFID'
    
RFID_file_names = os.listdir(RFID_path)

RFID_data_dir_paths = []

for file in RFID_file_names:
    int_path = RFID_path + '/' + file
    RFID_data_dir_paths.append(int_path)
    
RFID_data_dir_paths

['/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/RFID/.DS_Store',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/RFID/Idspyder-21.06.17.csv']

#### Clean df and combine into 1

In [12]:
data_RFID = pd.DataFrame()

for path in RFID_data_dir_paths:
    print(path)
    
    if path.split('/')[-1] == '.DS_Store':
        continue
        
    #create data table
    data_RFID_int = pd.read_csv(path, sep=';', encoding='utf-16')
    data_RFID_int = pd.DataFrame(data = data_RFID_int)
    data_RFID_int = data_RFID_int.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

    print(data_RFID_int.shape)
    
    #drop rows with no event duration
    data_RFID_int = data_RFID_int[~data_RFID_int['eventDuration'].isna()]
    
    #convert from ms to sec
    data_RFID_int['eventDuration'] = data_RFID_int['eventDuration']/1000
    #clean up and create datetime columns
    data_RFID_int['RFID_datetime_orig'] = pd.to_datetime(data_RFID_int['DateTime'], unit='d')
    #can't figure out datetime format so hard code for now
    data_RFID_int['RFID_datetime_start'] = [x.replace(year = 2021) - dt.timedelta(days = 2) for x in data_RFID_int['RFID_datetime_orig']] 
    data_RFID_int['RFID_date'] = data_RFID_int['RFID_datetime_start'].dt.date 
    data_RFID_int['RFID_day'] = data_RFID_int['RFID_datetime_start'].dt.day 
    data_RFID_int['RFID_time_start'] = [x.split(' ')[1] for x in data_RFID_int['RFID_datetime_start'].dt.strftime("%Y-%m-%d %H:%M:%S")]
    data_RFID_int['RFID_hour'] = data_RFID_int['RFID_datetime_start'].dt.hour
    data_RFID_int['RFID_min'] = data_RFID_int['RFID_datetime_start'].dt.minute
    data_RFID_int['RFID_second'] = data_RFID_int['RFID_datetime_start'].dt.second

    if data_RFID.empty:
        data_RFID = data_RFID_int
    else:
        data_RFID = pd.concat([data_RFID, data_RFID_int], axis=0)
        
    print(data_RFID.shape,'\n')

#clean columns
data_RFID = data_RFID[['IdRFID', 'unitLabel', 'eventDuration',
       'RFID_datetime_start', 'RFID_date',
       'RFID_day', 'RFID_time_start', 'RFID_hour', 'RFID_min', 'RFID_second']]
    
data_RFID.head()

/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/RFID/.DS_Store
/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/RFID/Idspyder-21.06.17.csv
(31705, 14)
(31704, 22) 



,IdRFID,unitLabel,eventDuration,RFID_datetime_start,RFID_date,RFID_day,RFID_time_start,RFID_hour,RFID_min,RFID_second
17,041A555D92,RFID8,8.064,2021-06-17 08:37:05.187993600,2021-06-17,17,08:37:05,8,37,5
18,041A555D92,RFID8,2.176,2021-06-17 08:37:15.566016000,2021-06-17,17,08:37:15,8,37,15
19,041A555D92,RFID8,0.075,2021-06-17 08:37:20.556998400,2021-06-17,17,08:37:20,8,37,20
20,041A555D92,RFID8,4.705,2021-06-17 08:37:22.770998400,2021-06-17,17,08:37:22,8,37,22
21,041A555D92,RFID8,1.760,2021-06-17 08:37:29.090035200,2021-06-17,17,08:37:29,8,37,29


#### Compute endtime 

In [13]:
#compute end time for each eventDuration
def add_duration(a, b):
    newtime = a + dt.timedelta(0, b)

    return newtime

data_RFID['RFID_datetime_end'] = data_RFID.apply(lambda row: add_duration(row['RFID_datetime_start'], row['eventDuration']), axis=1)
data_RFID['RFID_time_end'] = [x.split(' ')[1] for x in data_RFID['RFID_datetime_end'].dt.strftime("%Y-%m-%d %H:%M:%S")]

print(data_RFID.shape)
data_RFID.head()

(31704, 12)


,IdRFID,unitLabel,eventDuration,RFID_datetime_start,RFID_date,RFID_day,RFID_time_start,RFID_hour,RFID_min,RFID_second,RFID_datetime_end,RFID_time_end
17,041A555D92,RFID8,8.064,2021-06-17 08:37:05.187993600,2021-06-17,17,08:37:05,8,37,5,2021-06-17 08:37:13.251993600,08:37:13
18,041A555D92,RFID8,2.176,2021-06-17 08:37:15.566016000,2021-06-17,17,08:37:15,8,37,15,2021-06-17 08:37:17.742016000,08:37:17
19,041A555D92,RFID8,0.075,2021-06-17 08:37:20.556998400,2021-06-17,17,08:37:20,8,37,20,2021-06-17 08:37:20.631998400,08:37:20
20,041A555D92,RFID8,4.705,2021-06-17 08:37:22.770998400,2021-06-17,17,08:37:22,8,37,22,2021-06-17 08:37:27.475998400,08:37:27
21,041A555D92,RFID8,1.760,2021-06-17 08:37:29.090035200,2021-06-17,17,08:37:29,8,37,29,2021-06-17 08:37:30.850035200,08:37:30


#### Add next column (do for each unitLabel separately)

In [ ]:
#column for if next is same Id
def next_compare(a, b):
    if a == b:
        compare = 'same'
        
    if a != b:
        compare = 'diff'

    return compare

data_RFID_next = pd.DataFrame()

for unitlabel in data_RFID['unitLabel'].unique():
    print(unitlabel)

    data_int = data_RFID[data_RFID['unitLabel'] == unitlabel].sort_values(['RFID_datetime_start'])
    data_int['RFID_datetime_start_next'] = data_int['RFID_datetime_start'].shift(-1)
    data_int['IdRFID_next'] = data_int['IdRFID'].shift(-1)
    
    data_int['IdRFID_diff_next'] = data_int.apply(lambda row: next_compare(row['IdRFID'], row['IdRFID_next']), axis=1)

    if data_RFID_next.empty:
        data_RFID_next = data_int
    else:
        data_RFID_next = pd.concat([data_RFID_next, data_int], axis=0)

data_RFID_next['time_diff_next'] = (data_RFID_next['RFID_datetime_start_next'] - data_RFID_next['RFID_datetime_end'])
data_RFID_next['time_diff_next'] = [x.total_seconds() for x in data_RFID_next['time_diff_next']]
data_RFID_next['time_diff_next_round'] = data_RFID_next['time_diff_next'].round(-1)
data_RFID_next.head()

#### Add day, hour, and min counts

In [ ]:
#column of running day count
dat_map = dict(zip(data_RFID_next['RFID_date'].unique(), np.arange(1,len(data_RFID_next['RFID_date'].unique())+1)))
data_RFID_next['RFID_day_count'] = data_RFID_next['RFID_date'].map(dat_map)

#column of running min and second count within each day
data_RFID_next['RFID_min_count'] = (data_RFID_next['RFID_hour']*60) + data_RFID_next['RFID_min'] 
data_RFID_next['RFID_sec_count'] = (data_RFID_next['RFID_min_count']*60) + data_RFID_next['RFID_second'] 

#column of running hour and minute count across all days
data_RFID_next['RFID_hour_count_running'] = (data_RFID_next['RFID_day_count']*24) + data_RFID_next['RFID_hour'] - 24
data_RFID_next['RFID_min_count_running'] = (data_RFID_next['RFID_hour_count_running']*60) + data_RFID_next['RFID_min'] 
data_RFID_next['RFID_sec_count_running'] = (data_RFID_next['RFID_min_count_running']*60) + data_RFID_next['RFID_second'] - 60

print(data_RFID_next.shape)
data_RFID_next.head()

#### Add light dark

In [ ]:
def assign_ld(time):
    if (time >= 6) & (time < 18):
        return 'light'
    if (time >= 0) & (time < 6):
        return 'dark_am'
    if (time >= 18):
        return 'dark_pm'
    
data_RFID_next['ld_seg'] = data_RFID_next['RFID_hour'].apply(assign_ld)

print(data_RFID_next.shape)
data_RFID_next.head()


#### Add day/night mapping (lines up dates and ld segments)

In [ ]:
ld_mapping_path = '/Users/abbieschindler/Documents/ProgrammingFun/iPythonScripts/ld_mapping.xlsx'

ld_mapping = pd.read_excel(ld_mapping_path)
ld_mapping = pd.DataFrame(data = ld_mapping)
print(ld_mapping.shape)
ld_mapping.head()

In [ ]:
data_RFID_next = data_RFID_next.merge(ld_mapping, right_on=['day', 'ld_seg'], left_on=['RFID_day_count', 'ld_seg'])
print(data_RFID_next.shape)
data_RFID_next.head()

#### Map unitLabel and bottle contents

In [ ]:
unitLabel_map = {'RFID5': 'RFID_EtOH10',
                 'RFID6': 'RFID_Suc2',
                 'RFID7': 'RFID_EtOH20',
                 'RFID8': 'RFID_H20',
                 'AM1-5': 'RFID_EtOH10',
                 'AM1-6': 'RFID_Suc2',
                 'AM1-7': 'RFID_EtOH20',
                 'AM1-8': 'RFID_H20',
                 'AM1-9': 'RFID_EtOH10',
                 'AM1-10': 'RFID_Suc2',
                 'AM1-11': 'RFID_EtOH20',
                 'AM1-12': 'RFID_H20',
                 'AM1-13': 'RFID_EtOH10',
                 'AM1-14': 'RFID_Suc2',
                 'AM1-15': 'RFID_EtOH20',
                 'AM1-16': 'RFID_H20',}

data_RFID_next['unitLabel_drink'] = data_RFID_next['unitLabel'].map(unitLabel_map)
print(data_RFID_next.shape)

### VDM data

#### Make and clean df

In [ ]:
#create list of dir paths
VDM_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID/build_2.0/r3.intruders/VDM'
    
VDM_file_names = os.listdir(VDM_path)

VDM_data_dir_paths = []

for file in VDM_file_names:
    int_path = VDM_path + '/' + file
    VDM_data_dir_paths.append(int_path)
    
VDM_data_dir_paths

In [ ]:
data_VDM = pd.DataFrame()

for path in VDM_data_dir_paths:
    print(path)
    
    if path.split('/')[-1] == '.DS_Store':
        continue
        
    #create data table
    data_VDM_int = pd.read_table(path, sep=';')
    data_VDM_int = pd.DataFrame(data = data_VDM_int)
    data_VDM_int = data_VDM_int.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])['Columbus Instruments Device Interface Version 1.5'].str.split(',', expand=True)
    data_VDM_int.columns = data_VDM_int.loc[18].values
    data_VDM_int = data_VDM_int.drop([18])
    
    #data_VDM_int['Cohort'] = path.split('/')[-2].split('.')[0]
    #data_VDM_int['TP'] = path.split('/')[-2].split('.')[1]
    #data_VDM_int['StartDate'] = path.split('/')[-1].split('.')[0]

    if data_VDM.empty:
        data_VDM = data_VDM_int
    else:
        data_VDM = pd.concat([data_VDM, data_VDM_int], axis=0)
        
    print(data_VDM.shape,'\n')

print(data_VDM.shape)
data_VDM.head()

#### Clean up column names

In [ ]:
data_VDM.columns = ['Int', 'Cage_VDM', 'VDM_Time', 
                    'VDM_EtOH10', 'VDM_EtOH10_accum',
                    'VDM_Suc2', 'VDM_Suc2_accum', 
                    'VDM_EtOH20', 'VDM_EtOH20_accum', 
                    'VDM_H20', 'VDM_H20_accum', '']

#data_VDM = data_VDM[['Cage_VDM', 'Cohort', 'TP', 'StartDate', 'VDM_Time', 'VDM_EtOH10', 'VDM_EtOH20', 'VDM_Suc2', 'VDM_H20']]

data_VDM.tail()

#### Convert liquid vol to etoh vol

In [ ]:
data_VDM['VDM_EtOH10'] = data_VDM['VDM_EtOH10'].astype('float')
data_VDM['VDM_EtOH20'] = data_VDM['VDM_EtOH20'].astype('float')
data_VDM['VDM_Suc2'] = data_VDM['VDM_Suc2'].astype('float')
data_VDM['VDM_H20'] = data_VDM['VDM_H20'].astype('float')

#### Make into tidy df

In [ ]:
#melt dataframe (new column of parameters)

meta_params = ['Cage_VDM', 'VDM_Time']
intake_params = ['VDM_EtOH10', 'VDM_EtOH20', 'VDM_Suc2', 'VDM_H20']

data_VDM_tidy = pd.melt(data_VDM, id_vars=meta_params, value_vars=intake_params)

print(data_VDM_tidy.shape)
data_VDM_tidy.head()

#### Drop times with no intake data

In [ ]:
data_VDM_tidy_intake = data_VDM_tidy[data_VDM_tidy['value'] > 0]
print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

#### Clean up datetime

In [ ]:
#clean up and create datetime columns
data_VDM_tidy_intake['VDM_datetime'] = pd.to_datetime(data_VDM_tidy_intake['VDM_Time'])
data_VDM_tidy_intake['VDM_timestamp'] = [x.timestamp() for x in data_VDM_tidy_intake['VDM_datetime']]

data_VDM_tidy_intake['VDM_date'] = data_VDM_tidy_intake['VDM_datetime'].dt.date
data_VDM_tidy_intake['VDM_day'] = data_VDM_tidy_intake['VDM_datetime'].dt.day
data_VDM_tidy_intake['VDM_time'] = data_VDM_tidy_intake['VDM_datetime'].dt.strftime("%H:%M:%S")
data_VDM_tidy_intake['VDM_hour'] = data_VDM_tidy_intake['VDM_datetime'].dt.hour
data_VDM_tidy_intake['VDM_min'] = data_VDM_tidy_intake['VDM_datetime'].dt.minute
data_VDM_tidy_intake['VDM_second'] = data_VDM_tidy_intake['VDM_datetime'].dt.second

print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

#### Add light dark and clean

In [ ]:
def assign_ld(time):
    if (time >= 6) & (time < 18):
        return 'light'
    if (time >= 0) & (time < 6):
        return 'dark_am'
    if (time >= 18):
        return 'dark_pm'
    
data_VDM_tidy_intake['ld_seg'] = data_VDM_tidy_intake['VDM_hour'].apply(assign_ld)

data_VDM_tidy_intake.head()

In [ ]:
#column of running day count
dat_map = dict(zip(data_VDM_tidy_intake['VDM_date'].sort_values().unique(), np.arange(1,len(data_VDM_tidy_intake['VDM_date'].unique())+1)))
data_VDM_tidy_intake['day_count'] = data_VDM_tidy_intake['VDM_date'].map(dat_map)

#column of running min and second count within each day
data_VDM_tidy_intake['min_count'] = (data_VDM_tidy_intake['VDM_hour']*60) + data_VDM_tidy_intake['VDM_min'] 
data_VDM_tidy_intake['sec_count'] = (data_VDM_tidy_intake['min_count']*60) + data_VDM_tidy_intake['VDM_second'] 

#column of running hour and minute count across all days
data_VDM_tidy_intake['hour_count_running'] = (data_VDM_tidy_intake['day_count']*24) + data_VDM_tidy_intake['VDM_hour'] - 24
data_VDM_tidy_intake['min_count_running'] = (data_VDM_tidy_intake['hour_count_running']*60) + data_VDM_tidy_intake['VDM_min'] 
data_VDM_tidy_intake['sec_count_running'] = (data_VDM_tidy_intake['min_count_running']*60) + data_VDM_tidy_intake['VDM_second'] - 60

print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:
data_arduino.head()

#### Viz

In [ ]:
data_RFID_viz = data_RFID_next[data_RFID_next['unitLabel_drink'] == 'RFID_H20']
data_RFID_viz = data_RFID_viz[data_RFID_viz['RFID_day_count'] == 1]

data_VDM_viz = data_VDM_tidy_intake[data_VDM_tidy_intake['variable'] == 'VDM_H20']
data_VDM_viz = data_VDM_viz[data_VDM_viz['VDM_day_count'] == 1]

data_arduino_viz = data_arduino[data_arduino['unitLabel_drink'] == 'arduino_H20']
data_arduino_viz = data_arduino_viz[data_arduino_viz['arduino_day_count'] == 1]

colorCodes = ['C{}'.format(i) for i in range(3)]

data_1 = data_RFID_viz['RFID_min_count_running'].values
data_2 = data_VDM_viz['min_count_running'].values
data_3 = data_arduino_viz['arduino_min_count_running'].values
#data_4 = data_arduino[data_arduino['device'] == 'A3']['arduino_sec_count_running'].values

data_array = np.array([data_1, data_2, data_3])

# Draw a spike raster plot
plt.figure(figsize=(50,15))
params = {'axes.labelsize': 75,
          'axes.titlesize': 75}
plt.rc('xtick', labelsize=40) 
plt.rc('ytick', labelsize=40) 
plt.rcParams.update(params)
plt.yticks([0,1,2], 
           ['RFID', 'VDM', 'arduino', ])
plt.eventplot(data_array, color=colorCodes, linelengths = 0.3) 
plt.xlabel('Time (running second count)')
plt.ylabel('Animal and group number')


plt.show()

### Filter RFID data by VDM times

In [ ]:
data_VDM_final = data_VDM_tidy_intake.sort_values(['variable', 'VDM_datetime']).reset_index(drop=True)
data_VDM_final = data_VDM_final[data_VDM_final['Cage_VDM'] == '2']
print(data_VDM_final.shape)
data_VDM_final.head()

In [ ]:
data_RFID_final= data_RFID_next.sort_values(['unitLabel_drink', 'RFID_datetime_start']).reset_index(drop=True)
data_RFID_final.head()

In [ ]:
data_arduino_final= data_arduino.sort_values(['unitLabel_drink', 'arduino_datetime']).reset_index(drop=True)
data_arduino_final.head()

#### RFID and arduino

In [ ]:
arduino_data = data_arduino_final
RFID_data = data_RFID_final

arduino_RFID_mapping = {'arduino_EtOH10': 'RFID_EtOH10',
                    'arduino_Suc2': 'RFID_Suc2',
                    'arduino_EtOH20': 'RFID_EtOH20',
                    'arduino_H20': 'RFID_H20'}

RFID_eventDuration = []
RFID_assign_id_pre = []
RFID_assign_var_pre = []
RFID_datetime_start_pre = []
RFID_datetime_end_pre = []
RFID_datetime_start_next = []
IdRFID_next = []
IdRFID_diff_next = []
time_diff_next = []
ld = []

for index, row in arduino_data.iterrows():
    if index%1000 == 0:
        print(index)
    #find RFID signal start time closest to VDM time before
    try:
        int_assign = RFID_data[(RFID_data['unitLabel_drink'] == arduino_RFID_mapping[row['unitLabel_drink']]) & #match tube
                               (RFID_data['RFID_datetime_start'] <= row['arduino_datetime']) #find closest datetime
                              ].sort_values(['RFID_datetime_start']).iloc[-1]

        RFID_eventDuration.append(int_assign['eventDuration'])
        RFID_assign_id_pre.append(int_assign['IdRFID'])
        RFID_assign_var_pre.append(int_assign['unitLabel'])
        RFID_datetime_start_pre.append(int_assign['RFID_datetime_start'])
        RFID_datetime_end_pre.append(int_assign['RFID_datetime_end'])
        RFID_datetime_start_next.append(int_assign['RFID_datetime_start_next'])
        IdRFID_next.append(int_assign['IdRFID_next'])
        IdRFID_diff_next.append(int_assign['IdRFID_diff_next'])
        time_diff_next.append(int_assign['time_diff_next'])
        ld.append(int_assign['ld'])
        
    except:
        print(index, 'pre')
        RFID_eventDuration.append(np.nan)
        RFID_assign_id_pre.append(np.nan)
        RFID_assign_var_pre.append(np.nan)
        RFID_datetime_start_pre.append(np.nan)
        RFID_datetime_end_pre.append(np.nan)
        RFID_datetime_start_next.append(np.nan)
        IdRFID_next.append(np.nan)
        IdRFID_diff_next.append(np.nan)
        time_diff_next.append(np.nan)
        ld.append(np.nan)
        
arduino_data['RFID_eventDuration'] = RFID_eventDuration
arduino_data['RFID_assign_id_pre'] = RFID_assign_id_pre
arduino_data['RFID_assign_var_pre'] = RFID_assign_var_pre
arduino_data['RFID_datetime_start_pre'] = RFID_datetime_start_pre
arduino_data['RFID_datetime_end_pre'] = RFID_datetime_end_pre
arduino_data['RFID_datetime_start_next'] = RFID_datetime_start_next
arduino_data['IdRFID_next'] = IdRFID_next
arduino_data['IdRFID_diff_next'] = IdRFID_diff_next
arduino_data['time_diff_next'] = time_diff_next
arduino_data['ld'] = ld

print(arduino_data.shape)
arduino_data.head()

In [ ]:
def time_within(RFID_start, RFID_stop, VDM_time):
    
    if (VDM_time >= RFID_start) & (VDM_time <= RFID_stop):
        return 0
    
    elif (VDM_time >= RFID_start) & (VDM_time >= RFID_stop):
        time_within = VDM_time - RFID_stop
        time_within = time_within.total_seconds()
        return time_within
    
arduino_data['arduino_RFID_timediff'] = arduino_data.apply(lambda row: time_within(row['RFID_datetime_start_pre'], 
                                                                   row['RFID_datetime_end_pre'],
                                                                   row['arduino_datetime']), axis=1)

arduino_data.head()

In [ ]:
data_arduino_final['arduino_RFID_timediff'].mean()

#### RFID and VDM

In [ ]:
VDM_data = data_VDM_final
RFID_data = data_RFID_final

VDM_RFID_mapping = {'VDM_EtOH10': 'RFID_EtOH10',
                    'VDM_Suc2': 'RFID_Suc2',
                    'VDM_EtOH20': 'RFID_EtOH20',
                    'VDM_H20': 'RFID_H20'}

RFID_eventDuration = []
RFID_assign_id_pre = []
RFID_assign_var_pre = []
RFID_datetime_start_pre = []
RFID_datetime_end_pre = []
RFID_datetime_start_next = []
IdRFID_next = []
IdRFID_diff_next = []
time_diff_next = []
ld = []

for index, row in VDM_data.iterrows():
    if index%1000 == 0:
        print(index)
    #find RFID signal start time closest to VDM time before
    try:
        int_assign = RFID_data[(RFID_data['unitLabel_drink'] == VDM_RFID_mapping[row['variable']]) & #match tube
                               (RFID_data['RFID_datetime_start'] <= row['VDM_datetime']) #find closest datetime
                              ].sort_values(['RFID_datetime_start']).iloc[-1]

        RFID_eventDuration.append(int_assign['eventDuration'])
        RFID_assign_id_pre.append(int_assign['IdRFID'])
        RFID_assign_var_pre.append(int_assign['unitLabel'])
        RFID_datetime_start_pre.append(int_assign['RFID_datetime_start'])
        RFID_datetime_end_pre.append(int_assign['RFID_datetime_end'])
        RFID_datetime_start_next.append(int_assign['RFID_datetime_start_next'])
        IdRFID_next.append(int_assign['IdRFID_next'])
        IdRFID_diff_next.append(int_assign['IdRFID_diff_next'])
        time_diff_next.append(int_assign['time_diff_next'])
        ld.append(int_assign['ld'])
        
    except:
        print(index, 'pre')
        RFID_eventDuration.append(np.nan)
        RFID_assign_id_pre.append(np.nan)
        RFID_assign_var_pre.append(np.nan)
        RFID_datetime_start_pre.append(np.nan)
        RFID_datetime_end_pre.append(np.nan)
        RFID_datetime_start_next.append(np.nan)
        IdRFID_next.append(np.nan)
        IdRFID_diff_next.append(np.nan)
        time_diff_next.append(np.nan)
        ld.append(np.nan)
        
VDM_data['RFID_eventDuration'] = RFID_eventDuration
VDM_data['RFID_assign_id_pre'] = RFID_assign_id_pre
VDM_data['RFID_assign_var_pre'] = RFID_assign_var_pre
VDM_data['RFID_datetime_start_pre'] = RFID_datetime_start_pre
VDM_data['RFID_datetime_end_pre'] = RFID_datetime_end_pre
VDM_data['RFID_datetime_start_next'] = RFID_datetime_start_next
VDM_data['IdRFID_next'] = IdRFID_next
VDM_data['IdRFID_diff_next'] = IdRFID_diff_next
VDM_data['time_diff_next'] = time_diff_next
VDM_data['ld'] = ld

print(VDM_data.shape)
VDM_data.head()

In [ ]:
def time_within(RFID_start, RFID_stop, VDM_time):
    
    if (VDM_time >= RFID_start) & (VDM_time <= RFID_stop):
        return 0
    
    elif (VDM_time >= RFID_start) & (VDM_time >= RFID_stop):
        time_within = VDM_time - RFID_stop
        time_within = time_within.total_seconds()
        return time_within
    
VDM_data['VDM_RFID_timediff'] = VDM_data.apply(lambda row: time_within(row['RFID_datetime_start_pre'], 
                                                                   row['RFID_datetime_end_pre'],
                                                                   row['VDM_datetime']), axis=1)

VDM_data.head()

In [ ]:
VDM_data['VDM_RFID_timediff'].mean()

#### Add metadata

In [ ]:
data_comb = meta_data.merge(data_comb.reset_index(), left_on='IdRFID', right_on='RFID_assign_id_pre', sort=False)

print(data_comb.shape)
data_comb.head()

#### Add day, hour, and min counts

In [ ]:
#column of running day count
dat_map = dict(zip(data_comb['VDM_date'].sort_values().unique(), np.arange(1,len(data_comb['VDM_date'].unique())+1)))
data_comb['day_count'] = data_comb['VDM_date'].map(dat_map)

#column of running min and second count within each day
data_comb['min_count'] = (data_comb['VDM_hour']*60) + data_comb['VDM_min'] 
data_comb['sec_count'] = (data_comb['min_count']*60) + data_comb['VDM_second'] 

#column of running hour and minute count across all days
data_comb['hour_count_running'] = (data_comb['day_count']*24) + data_comb['VDM_hour'] - 24
data_comb['min_count_running'] = (data_comb['hour_count_running']*60) + data_comb['VDM_min'] 
data_comb['sec_count_running'] = (data_comb['min_count_running']*60) + data_comb['VDM_second'] - 60

print(data_comb.shape)
data_comb.head()

#### Viz

In [ ]:
plt.figure(figsize=(10,10))
params = {'axes.labelsize': 30,
          'axes.titlesize': 30}
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20) 
plt.rcParams.update(params)

#data_int = data_comb[data_comb['VDM_hour']==18]
sns.displot(data=data_comb, x="VDM_RFID_timediff", kind="ecdf", hue='variable')
#plt.xlim(0, 30)
plt.xlabel('Time (running second count)')
plt.ylabel('Task')
plt.show()

### Export data

In [ ]:
data_RFID_final.to_csv('data_RFID_final_1m_suc.csv')
data_comb.to_csv('data_comb_1m_suc.csv')

In [ ]:
data_RFID_viz = data_RFID_final[data_RFID_final['unitLabel_drink'] == 'RFID_H20']
data_RFID_viz = data_RFID_final[data_RFID_final['RFID_min_count_running'] == 506]

data_arduino_viz = data_arduino[data_arduino['arduino_min_count_running'] == 506]
data_VDM_viz = data_VDM_final[data_VDM_final['min_count_running'] == 506]

colorCodes = ['C{}'.format(i) for i in range(5)]

data_1 = data_RFID_viz[data_RFID_viz['IdRFID'] == '041A555C29']['RFID_sec_count_running'].values
data_2 = data_RFID_viz[data_RFID_viz['IdRFID'] == '041A5553B4']['RFID_sec_count_running'].values
data_3 = data_RFID_viz[data_RFID_viz['IdRFID'] == '041A555D92']['RFID_sec_count_running'].values

data_4 = data_arduino_viz[data_arduino_viz['device'] == 'A3']['arduino_sec_count_running'].values
data_5 = data_VDM_viz[data_VDM_viz['variable'] == 'VDM_H20']['sec_count_running'].values

data_array = np.array([data_1, data_2, data_3, data_4, data_5])

# Draw a spike raster plot
plt.figure(figsize=(50,15))
params = {'axes.labelsize': 75,
          'axes.titlesize': 75}
plt.rc('xtick', labelsize=40) 
plt.rc('ytick', labelsize=40) 
plt.rcParams.update(params)
plt.yticks([0,1,2,3,4], 
           ['RFID1', 'RFID2', 'RFID3', 'arduino', 'VDM'])
plt.eventplot(data_array, color=colorCodes, linelengths = 0.3) 
plt.xlabel('Time (running second count)')
plt.ylabel('Animal and group number')


plt.show()